In [1]:
import sys

from exceptiongroup import catch

print(f'Python {sys.version}') 

Python 3.8.10 (default, Jul 29 2024, 17:02:10) 
[GCC 9.4.0]


In [2]:
import json
import os

page_file = 'resources/pages/letter.json'
json_data = None

if os.path.exists(page_file):
    with open(page_file, 'r') as f:
        json_data = json.load(f)

if json_data is not None and 'page' in json_data:
    print(json_data["page"]["dpi"])
    


[300, 300]


In [4]:
import json
from PIL import Image, ImageDraw

class GraphicalPage:
    def __init__(self, width_in, height_in, dpi, bg_color, border_width, border_color):
        # Convert dimensions from inches to pixels using DPI
        self.width_px = int(width_in * dpi[0])
        self.height_px = int(height_in * dpi[1])
        self.dpi = dpi
        self.bg_color = bg_color
        self.border_width = border_width
        self.border_color = border_color

    @classmethod
    def from_json(cls, json_data):
        # Parse the JSON data
        page_data = json_data['page']
        dimensions = page_data['dimensions']
        dpi = page_data['dpi']
        bg_color = page_data['bg']
        border_data = page_data['border']
        border_width = border_data['width']
        border_color = border_data['color']

        return cls(
            width_in=dimensions['width'],
            height_in=dimensions['height'],
            dpi=dpi,
            bg_color=bg_color,
            border_width=border_width,
            border_color=border_color
        )

    def create_image(self):
        # Create a blank image with the background color
        image = Image.new("RGB", (self.width_px, self.height_px), self.bg_color)
        draw = ImageDraw.Draw(image)

        # Draw the border if border width is greater than 0
        if self.border_width > 0:
            # Calculate the border dimensions and draw the rectangle
            draw.rectangle(
                [(self.border_width // 2, self.border_width // 2), 
                 (self.width_px - self.border_width // 2, self.height_px - self.border_width // 2)], 
                outline=self.border_color, 
                width=self.border_width
            )

        return image

# Example usage:

page_data = None
# Load JSON data
if os.path.exists(page_file):
    with open(page_file, 'r') as f:
        page_data = json.load(f)

# page_data = json.loads(json_string)

# Create the graphical page from JSON
page = GraphicalPage.from_json(page_data)

# Generate the image
image = page.create_image()

# Save or display the image
image.show()  # This will open the image in the default viewer
image.save("output_page.png")  # Optionally save the image to a file


In [2]:
from mememaker import GraphicalPage

# Create the graphical page from JSON
page = GraphicalPage.load_from_name("hd")

# Generate the image
image = page.create_image()

# Save or display the image
image.show()  # This will open the image in the default viewer
image.save("output_page.png")  #

In [7]:
from mememaker import GraphicalPage, load_template

page_file = 'resources/pages/letter.json'

# Create the graphical page from JSON
page = GraphicalPage.load_from_filename(page_file)

# Generate the image
image = page.create_image()

# Save or display the image
image.show()  # This will open the image in the default viewer
image.save("output_page.png")  #


In [14]:
from mememaker import scan_directory_for_json

# Example usage
path_list = [ 'resources/pages', 'resources/templates', 'resources/ridiculous' ]

for directory_path in path_list:
    try:
        json_files = scan_directory_for_json(directory_path)
    
        # Output the list of .json files found
        for json_file in json_files:
            print(json_file)
    except Exception as e:
        print(f'Error : {e}')
        

resources/pages/hd.json
resources/pages/letter.json
resources/pages/a4.json
resources/templates/template.json
Error : No JSON files found in directory: resources/ridiculous


In [11]:
from mememaker import load_page, load_template
import json

try:
    page_json = load_page("a4")
    template_json = load_template("meme1")
    
    print(json.dumps(page_json, indent=2))
    print(json.dumps(template_json, indent=2))
except Exception as e:
    print(f'Error: {e}')


{
  "page": {
    "name": "a4",
    "dimensions": {
      "width": 8.27,
      "height": 11.69,
      "units": "in"
    },
    "dpi": [
      300,
      300
    ],
    "bg": "lightblue",
    "border": {
      "width": 8,
      "color": "yellow"
    }
  }
}
{
  "template": {
    "name": "meme1",
    "page": "letter",
    "areas": [
      {
        "name": "image",
        "type": "image",
        "dimensions": {
          "top": 20,
          "left": 10,
          "bottom": 80,
          "right": 90
        }
      }
    ]
  }
}


## Example using Exception

In [6]:
from mememaker import load_page, load_template
import json

try:
    page_dats = load_page('shit')
except FileNotFoundError as e:
    print(f"Error: {e}")
    

Error: No page found with the name 'shit' in resources/pages


In [1]:
from mememaker import get_page_names, get_template_names

page_names = get_page_names()
template_names = get_template_names()

print(page_names)
print(template_names)


['hd', 'letter', 'a4']
['meme1']
